In [1]:
import logging

In [2]:
def print_graph(graph):
    for k, v in graph.iteritems():        
        logging.debug('{}: {}'.format(k, v))

In [3]:
def duplicate_and_transform_graph(graph, T):
    l = len(graph)
    new_graph = {}
    offset = max(graph.keys()) + 1
    logging.debug('duplication offset: {}'.format(offset))
    to_duplicant = lambda x: None if x is None else x + offset
    for k, (formula, (cyclic, neighbours)) in graph.iteritems():
        new_graph[k] = formula, (cyclic, neighbours)
        new_graph[to_duplicant(k)] = T + formula, (cyclic, [to_duplicant(x) for x in neighbours])  
    return new_graph

In [4]:
from copy import copy
def clone_graph(graph):
    return {k: (f, (m, copy(ns))) for k, (f, (m, ns)) in graph.iteritems()}

In [5]:
# def apply_prefix_rules(graph, rules):
#     new_graph = []
#     for formula, (mod, neighbours) in graph:
#         for (prefix, replacement) in rules:
#             if formula.startswith(rule):
#                 new_formula = replacement + formula[len(prefix)+1:]
#             else:
#                 new_formula = formula
#             new_graph.append((new_formula, (mod, neighbours)))

In [6]:
# def intersection_with_indices(l1, l2):
#     common_members = set(l1).intersection(l2)
#     return [(x, l1.index(x), l2.index(x)) for x in common_members]

In [7]:
def pairwise_induction(pairs):
    for (a, b) in pairs:
        if a is not None and b is not None:
            yield (a, b)
        elif a is None:
            yield (b, b)
        else:
            yield (a, a)
            

In [8]:
def pad(l, n, padding_direction=1, v=None):
    from itertools import repeat
    if len(l) > l:
        return l
    padding = list(repeat(v, n - len(l)))
    if padding_direction > 0:
        return l + padding
    else:
        return padding + l

In [9]:
def has_repeating_values(l):
    without_nones = filter(lambda x: x is not None, l)
    return len(l) > len(set(l))

In [10]:
def is_sorted(l):
    return all([x < y for x, y in zip(l, l[1:])])

In [11]:
def padding_zip(l1, l2, padding_direction=1):
    length = max(len(l1), len(l2))
    return zip(pad(l1, length, padding_direction),
               pad(l2, length, padding_direction))

In [12]:
def mirror(tpls):
    if not tpls:
        return None
    (c, l) = tpls
    return (c, [(b, a) for (a, b) in l])

In [13]:
def common_cycle3(s1, s2):
    from itertools import repeat, chain
    (c1, l1) = s1
    (c2, l2) = s2
    if not c1 or not c2:
        #logging.debug( 'Non cyclic')
        return None
    if len(l1) != len(l2):
        #logging.debug( 'Non matching lengths {} {}'.format(l1, l2))
        return None
    common_members = set(l1).intersection(l2)
    if len(common_members) == 0 or common_members == {None}:
        logging.debug( 'No common members: {}, {}'.format(l1, l2))
        return None
    m0 = common_members.pop()
    skew1 = l1.index(m0)
    skew2 = l2.index(m0)
    candidate = list(pairwise_induction([(l1[(i + skew1) % len(l1)], l2[(i + skew2) % len(l2)])
                                         for i in range(len(l1))]))
    if any([x != y and (x in common_members or y in common_members) for x, y in candidate]):
        logging.debug( [x != y and (x in common_members or y in common_members) for x, y in candidate])
        logging.debug( 'Non matching cycles {}'.format(candidate))
        return None
    return (True, candidate)

In [14]:
def aget(l, i):
    return l[i] if i >= 0 and i < len(l) else None
def common_cycle2(s1, s2):
    import re
    from itertools import repeat, chain
    (c1, l1) = s1
    (c2, l2) = s2
    if len(l2) < len(l1) or (c1 and not c2):        
        return common_cycle2(s2, s1)
    # l1 <= l2, 
    # either c1 is open or c2 is closed
    common_numbers = set(l1).intersection(l2)
    def to_letter(n, wild='_'):
        if n > 26:
            raise Exception('Illegal ordinal: {}'.format(n))
        return chr(ord('A') + n) if n in common_numbers else wild
    def to_word(l):
        return ''.join([to_letter(n) for n in l])
    def to_result(match, reverse_lookup, force_closed=False, default=(None)):
        if not match:
            return default
        is_closed = force_closed or len(match.groups()) > 1
        return (is_closed, reverse_lookup[match.start():match.end()], match.re, match.string, match.group(0))
    if c1 and c2:
        # c1 is closed, c2 is closed
        regex = '({0})(?={1})'.format(to_word(l1), to_letter(l1[0]))
        matched_string = '{0}{0}'.format(to_word(l2), to_letter(l2[0]))
        reverse_lookup = l2 + l2
        return to_result(re.search(regex, matched_string), reverse_lookup, True)
    elif c2:
        # c1 is open, c2 is closed
        regex = '({0})({1}?)'.format(to_word(l1), to_letter(l1[0]))
        matched_string = '{0}{0}'.format(to_word(l2))
        reverse_lookup = l2 + l2
        match = re.search(regex, matched_string)
        return to_result(match, reverse_lookup)
    else:
        # c1 is open, c2 is open
        k1, fc1 = [(j, n) for j, n in enumerate(l1) if n in common_numbers][0]
        k2, fc2 = [(j, n) for j, n in enumerate(l2) if n in common_numbers][0]
        if fc1 != fc2:
            k1 = l1.index(fc2)
            while k2 - 1 >= 0 and \
                  k1 - 1 >= 0 and \
                  l1[k1 - 1] not in common_numbers and \
                  l2[k2 - 1] not in common_numbers:
                k1 = k1 - 1
                k2 = k2 - 1
            matched_string = to_word(l2)
            reverse_lookup = l2
            logging.debug( (k1, k2))
            regex = '{0}(_*){1}'.format(to_word(l1[k1:]), to_word(l1[0:k1]))
            return to_result(re.search(regex, matched_string), reverse_lookup, True)
        else:
            # Non cyclic result
            if k2 < k1:
                matched_string = to_word(l2)
                regex = to_word(l1[k1:])
                return (False, l1[0:k1] + l2[k2:])
            else:
                return (False, l2[0:k2] + l1[k1:])
#         closing_regex = '-*'.join([to_letter(n, wild='(_|-*)') for n in l1])
#         closing_matched_string = '{0}{1}{0}'.format(''.join([to_letter(n) for n in l2]),
#                                                     '-' * len(l1))
        
#         match = re.search(closing_regex, closing_matched_string)
#         if not match:
#             return (None, closing_regex, closing_matched_string)
#         if '-' in match.group(0):
#             if not re.search('-_+$', match.group(0)):
#                 # A cycle was closed by a common char
#                 a_common = list(common_numbers)[0]
#                 offset = l1.index(a_common) - l2.index(a_common)
#                 indices = range(0, match.end() - len(re.search('-*', match.group(0)).group(0)))
#                 firsts = [aget(l1, j - offset) for j in indices]
#                 seconds = [aget(l2, j) for j in indices]
#                 return (True, firsts, seconds, match.group(0), match.start(), match.end())
#             # A cycle was 'closed' by a wildcard, not considered a closing of a cycle
#             # Rather, the first sequence extends the second
#             indices = range(0, match.end() - len(re.search('-*', match.group(0)).group(0)))
#             offset = match.start()
#             firsts = [aget(l1, j - offset) for j in indices]
#             seconds = [aget(l2, j) for j in indices]
#             return (False, firsts, seconds, match.group(0), match.start(), match.end())
#         else:            
#             # First sequence is not conained in second
#             return (False, l2, match.group(0), match.start(), match.end())

In [15]:
def common_cycle(s1, s2):
    from itertools import repeat, chain
    (c1, l1) = s1
    (c2, l2) = s2
    if c1 and c2 and len(l1) != len(l2):
        logging.debug( 'Cannot combine closed cycles of different lengths')
    if has_repeating_values(l1):
        raise Exception('Non unique members in cycle 1')
    if has_repeating_values(l2):
        raise Exception('Non unique members in cycle 2')
    tpls = sorted(intersection_with_indices(l1, l2), key= lambda x: x[1:2])
    if len(tpls) == 0:
        logging.debug( 'Cannot combine non intersecting sections')
        return None
    if is_sorted([x[2] for x in tpls]):
        logging.debug( tpls)
        result = list(padding_zip(l1[0:tpls[0][1]], l2[0:tpls[0][2]], padding_direction=-1))
        for prv, tpl in zip(tpls, tpls[1:]):
            if tpl[1] - prv[1] == tpl[2] - prv[2]:
                result = result + list(zip(l1[prv[1]:tpl[1]], l2[prv[2]:tpl[2]]))
            else:
                logging.debug( 'Misatch: {} - {} != {} - {}'.format(tpl[1], prv[1], tpl[2], prv[2]))
                return None
        result = result + padding_zip(l1[tpls[-1][1]:], l2[tpls[-1][2]:])
        return (c1 or c2, list(pairwise_induction(result)))
    result = []
    overboard = False
    for i, prv, tpl in zip(range(len(tpls)), tpls, tpls[1:]):
        if tpl[2] < prv[2]:
            ''' List 2 performed a whole cycle '''
            if overboard:
                raise Exception('Ordering is not cyclic')
            overboard = True
            gap = (tpl[1] - prv[1]) - (tpl[2] + len(l2) - prv[2])
            if c2 and gap != 0:
                raise Exception('Mismatch in common index {} (when reaching end of l2, sorted)'.format(i))
            if gap < 0:
                raise Exception('Cannot negative pad in index {} (when reaching end of l2, sorted)'.format(i))
            s1 = [l1[i] for i in range(prv[1], tpl[1])]
            s2 = [l2[i] for i in range(prv[2], len(l2))] + \
                 list(repeat(None, gap)) + \
                 [l2[i] for i in range(0, tpl[2])]
            result = result + zip(s1, s2)
        elif tpl[2] - prv[2] != tpl[1] - prv[1]:
            raise Exception('Mismatch in index {} (sorted)'.format(i))
        else:
            s1 = [l1[i] for i in range(prv[1], tpl[1])]
            s2 = [l2[i] for i in range(prv[2], tpl[2])]
            result = result + zip(s1, s2)
    result_is_closed = c1 or c2 or overboard
    if result_is_closed:
        ''' List 1 must perform a whole cycle '''
        prv = tpls[-1]
        tpl = tpls[0]
        gap = (tpl[2] - prv[2]) - (tpl[1] + len(l1) - prv[1])
        if c1:
            if gap != 0:
                raise Exception('Mismatch in when reaching end of l1')
        if gap < 0:
            if tpl[2] < prv[2]:
                logging.debug( 'Gap : {}'.format(gap))
                result.append((l1[-1], l2[-1]))
            else:
                raise Exception('Cannot negative pad when reaching end of l1')
        else:
            s2 = [l2[i] for i in range(prv[2], tpl[2])]
            s1 = [l1[i] for i in range(prv[1], len(l1))] + \
                 list(repeat(None, gap)) + \
                 [l1[i] for i in range(0, tpl[1])]
            result = result + zip(s1, s2)
    else:
        result = padding_zip(l1[0:tpls[0][1]], l2[0:tpls[0][2]], padding_direction=-1) + \
                 result + \
                 padding_zip(l1[tpls[-1][1]:], l2[tpls[-1][2]:])
    return (result_is_closed, list(pairwise_induction(result)))                

In [16]:
def drop(n, it):
    for i in range(n):
        it.next()
    while True:
        yield it.next()

In [17]:
def tokenize_cycle(cycle, anchors):
    (cyclic, members) = cycle
    result = []
    k = 0
    for j in range(len(members)):
        if members[j] in anchors:
            result.append(members[k:j])
            result.append(members[j])
            k = j+1
    if cyclic:
        result[0] = members[k:] + result[0]
    else:
        result[0] = members[k:] + [-1] + result[0]
    return result
def padding_zip(l1, l2, tail=True):
    if len(l1) < len(l2):
        if tail:
            return zip(l1 + [None] * (len(l2) - len(l1)), l2)
        else: 
            return zip([None] * (len(l2) - len(l1)) + l1, l2)
    else:
        if tail:
            return zip(l1, l2 + [None] * (len(l1) - len(l2)))
        else:
            return zip(l1, [None] * (len(l1) - len(l2)) + l2)
def flexi_zip(l1, l2):
    if -1 in l1 and -1 in l2:
        idx1 = l1.index(-1)
        idx2 = l2.index(-1)        
        return padding_zip(l1[0:idx1], l2[0:idx2]) + padding_zip(l1[(idx1+1):], l2[(idx2+1):], False)
    if -1 in l1 and -1 not in l2:
        if (len(l2) - len(l1) + 1) < 0:
            return None
        idx = l1.index(-1)
        return zip(l1[0:idx] + [None] * (len(l2) - len(l1) + 1) + l1[idx+1:], l2)
    elif -1 in l2 and -1 not in l1:
        if (len(l1) - len(l2) + 1) < 0:
            return None
        idx = l2.index(-1)
        return zip(l1, l2[0:idx] + [None] * (len(l1) - len(l2) + 1) + l2[idx+1:])
    elif len(l1) == len(l2):
        return zip(l1, l2)
    else:
        return None
def common_cycle4(s1, s2):
    from itertools import repeat, chain
    (c1, l1) = s1
    (c2, l2) = s2
    common_members = set(s1[1]).intersection(s2[1]) - {None}
    if len(common_members) == 0:
        return None
    t1 = tokenize_cycle(s1, common_members)
    t2 = tokenize_cycle(s2, common_members)
    if len(t1) != len(t2):
        return None
    a_common_member = iter(common_members).next()
    idx1 = t1.index(a_common_member)
    idx2 = t2.index(a_common_member)
    result = []
    cyclic = True
    for j in range(len(t1)):
        token1 = t1[(j + idx1) % len(t1)]
        token2 = t2[(j + idx2) % len(t1)]
        if isinstance(token1, int):
            if not isinstance(token2, int):
                return None
            if token1 != token2:
                return None
            result.append((token1, token2))
            continue
        if isinstance(token2, int):
            return None
        if -1 in token1 and -1 in token2:
            cyclic = False
        fz = flexi_zip(token1, token2)
        if fz is None:
            return None
        result = result + fz
    return (cyclic, list(pairwise_induction(result)))
    

In [18]:
def compare_cyclic(l1, l2):
    from itertools import cycle
    if set(l1) != set(l2) or len(l1) != len(l2):
        return False
    for a,b in zip(l1, drop(l2.index(l1[0]), cycle(l2))):
        if a != b:
            return False
    return True

In [19]:
def index_or_identity(l, i):
    return l[i] if i in l else i

def without(l, indices):
    return [l[i] for i in range(len(l)) if i not in indices]

def without_keys(d, keys):
    return {k: v for k, v in d.iteritems() if k not in keys}

In [20]:
def common_member(l1, l2):    
    for i1 in range(len(l1)):
        x1 = l1[i1]
        if x1 in l2:
            return (i1, l2.index(l1[i1]))
    return (-1, -1)

In [21]:
def test(a, b, expected):
    cc = common_cycle4(a, b)
    (c, result) = cc
    rev = common_cycle4(b, a)
    (cm, mirrored_result) = rev
    (cme, mirrored_expected) = mirror(expected)
    if not compare_cyclic(result, expected[1]) or c != expected[0]:
        logging.debug( '{} and \n{} resulted in \n{} and did not result in \n{}'.format(a, b, cc, expected))
    if not compare_cyclic(mirrored_result, mirrored_expected) or cm != cme:
        logging.debug( '{} and \n{} resulted in \n{} and did not result in \n{}'.format(b, a, mirrored_result, mirror(expected)))        

In [22]:
test((False, [0,1]), (False, [1,2]),(False, [(0,0), (1,1), (2, 2)]))
test((False, [0,1,2,3]), (False, [2,3]),(False, [(0,0), (1,1), (2, 2), (3,3)]))
test((False, [0,1,2,3]), (False, [1,2]),(False, [(0,0), (1,1), (2, 2), (3,3)]))
test((False, [0,1,2,3]), (False, [1,None,3]),(False, [(0,0), (1,1), (2, 2), (3,3)]))
test((False, [0,1,2,3]), (False, [2,None,0]),(True, [(0,0), (1,1), (2, 2), (3,3)]))
test((False, [1,2]), (False, [2,3]),(False, [(1,1), (2,2), (3,3)]))
test((False, [0,1]), (True, [0,1]),(True, [(0,0), (1,1)]))
test((True, [11, 2, 0]), (False, [2, 0, 5]), (True, [(11, 5), (2, 2), (0, 0)]))

In [23]:
def replace(graph, merging_table):
    replacer = lambda x: merging_table[x] if x in merging_table else x
    return {
        k: (f, (c, map(replacer, n))) for k, (f, (c, n)) in graph.iteritems()
    }

def scan_for_errors(graph):
    for k, (f, (c, n)) in graph.iteritems():
        if len(n) != len(set(n)):
            print_graph(graph)
            raise Exception('Non unique members in node {}'.format(k))

In [40]:
from functools import partial
from copy import copy
def merge_vertices(graph, i1, i2, merging_table, countdown=100):
    if countdown < 0:
        logging.debug( 'Stack overflow')
        import sys
        sys.exit(1)
    logging.debug( 'Merging {i1} ({f1}) into {i2} ({f2})'.format(i1=i1, f1=graph[i1][0], i2=i2, f2=graph[i2][0]))
    logging.debug( 'with')
    logging.debug( 'Graph: ')
    print_graph(graph)
    logging.debug( 'Merging Table: ')
    logging.debug( merging_table)
    '''
    vertex of index i1 is merged into i2
    i1 - source index
    i2 - destination index
    '''
    if i1 == i2:
        logging.debug( 'Not merging')
        logging.debug( 'After merging {} into {}'.format(i1, i2))
        logging.debug( graph.keys())
        return graph, merging_table
    f1, (c1, n1) = graph[i1]
    f2, (c2, n2) = graph[i2]
    def mapper(l):
        return [merging_table.get(i, i) for i in l]    
    logging.debug('Common cycle of')
    logging.debug('{} , {}'.format(graph[i1], graph[i2]))
    logging.debug('{} , {}'.format((i1, c1, mapper(n1)), (i2, c2, mapper(n2))))
    cc = common_cycle((c1, mapper(n1)), (c2, mapper(n2)))
    logging.debug(cc)
    if cc is None:        
        logging.warn( 'After merging {} into {}'.format(i1, i2))
        logging.warn( graph.keys())
        raise Exception('Not merging')
    new_graph = clone_graph(graph)
    new_graph[i2] = (f2, (cc[0], [x[1] for x in cc[1]]))
    local_merging_table = {}
    for src, dst in cc[1]:
        if src != dst:
            local_merging_table[src] = dst
    new_merging_table = copy(local_merging_table)
    new_merging_table.update(merging_table)
    new_merging_table[i1] = i2
    for src, dst in local_merging_table.iteritems():
        if dst in new_graph and src in new_graph:
            new_graph, new_merging_table = merge_vertices(new_graph, src, dst, new_merging_table, countdown=countdown-1)
        else:
            logging.debug( "Skipping {} -> {}".format(src, dst))
#     logging.debug( 'combined_merging_table: {}'.format(combined_merging_table))
#     logging.debug( 'After merging {} into {}'.format(i1, i2))
#     logging.debug( without_keys(replace(new_graph, combined_merging_table), {i1}).keys())
    logging.debug('Returning')
    return (without_keys(replace(new_graph, new_merging_table), {i1}), new_merging_table)

In [25]:
import sys
handler = logging.StreamHandler(sys.stderr)
handler.level = logging.DEBUG
logger = logging.getLogger('')
logger.handlers = []
logger.setLevel(logging.DEBUG)
logging.getLogger('').addHandler(handler)

In [26]:
common_cycle3([True, [None, 1]], [True, [None, 7]])

No common members: [None, 1], [None, 7]


In [27]:
from collections import defaultdict

def progress_graph(initial_graph, path, rules):
    logging.warning('START')
    graph = initial_graph
    for c in path:
        logging.debug('Applying {}'.format(c))
        new_graph = duplicate_and_transform_graph(graph, c)
        logging.debug('Duplicated graph: ')
        print_graph(new_graph)
        
        for j in range(10):
            mutated = False
            grouped_by_formula = defaultdict(set)
            if j == 9:
                raise Exception('Endless loop')
            for k in sorted(new_graph.keys()):
                (f, x)  = new_graph[k]
                cnt = True
                new_formula = f
                while cnt:
                    brk = True
                    for fr, to in rules:
                        if fr in new_formula:
                            new_formula = new_formula.replace(fr, to)
                            brk = False
                            logging.debug( 'Reducing {} : {} into {} ({} -> {})'.format(k, f, new_formula, fr, to))
                    cnt = cnt and not brk
                grouped_by_formula[new_formula].add(k)
                new_graph[k] = (new_formula, x)    
            logging.debug( 'Reduced graph: ')
            print_graph(new_graph)
            merging_table = {}
            
            items = [(f, group) for (f, group) in grouped_by_formula.items() if len(group) > 1]
            logging.debug( 'Found the following groups: {}'.format(items) )
            if len(items) == 0:
                logging.debug('No groups, breaking')
                break
            for f, group in items:
                a = max(group)
                b = min(group)
                while b in merging_table:
                    b = merging_table[b]
                merging_table[a] = b

            logging.debug( 'Merging the following: {}'.format(merging_table) )
            new_merging_table = merging_table
            for src, dst in merging_table.iteritems():
                if dst in new_graph:
                    new_graph, new_merging_table = merge_vertices(new_graph, src, dst, new_merging_table)                    
                    #logging.debug( new_graph.keys())
                    if src not in new_graph:
                        mutated = True
                        logging.debug('mutated, {} not in graph'.format(src))
                else:
                    raise Exception( "Not merging {} -> {}".format(src, dst))                    
            graph = new_graph
            logging.debug( 'Result graph: ')
            print_graph(new_graph)
            if not mutated:
                logging.debug('Not mutated, breaking')
                break
    return graph

In [28]:
def normalize_indices(graph):
    keys = list(graph.keys())
    return {keys.index(k): [keys.index(n) for n in t[1][1]] for k, t in graph.items()}

In [48]:
#### actions = ['a', 'b']
cube_rules = [('aaa', ''),
         ('bbbb', ''),
         ('aP', 'P'),
         ('abP', 'bbbP')]


cube_lean_initial_graph = {
    0: ('P', (True, [1, 2, 3])),
    1: ('bP', (True, [0, None, None])),
    2: ('bbbP', (True, [0, None, None])),
    3: ('abbbP', (True, [0, None, None])),

}

cube_path = 'bb'
common_cycle = common_cycle3
logging.getLogger('').setLevel(logging.DEBUG)
progress_graph(cube_lean_initial_graph, cube_path, cube_rules)

START
Applying b
duplication offset: 4
Duplicated graph: 
0: ('P', (True, [1, 2, 3]))
1: ('bP', (True, [0, None, None]))
2: ('bbbP', (True, [0, None, None]))
3: ('abbbP', (True, [0, None, None]))
4: ('bP', (True, [5, 6, 7]))
5: ('bbP', (True, [4, None, None]))
6: ('bbbbP', (True, [4, None, None]))
7: ('babbbP', (True, [4, None, None]))
Reducing 6 : bbbbP into P (bbbb -> )
Reduced graph: 
0: ('P', (True, [1, 2, 3]))
1: ('bP', (True, [0, None, None]))
2: ('bbbP', (True, [0, None, None]))
3: ('abbbP', (True, [0, None, None]))
4: ('bP', (True, [5, 6, 7]))
5: ('bbP', (True, [4, None, None]))
6: ('P', (True, [4, None, None]))
7: ('babbbP', (True, [4, None, None]))
Found the following groups: [('P', set([0, 6])), ('bP', set([1, 4]))]
Merging the following: {4: 1, 6: 0}
Merging 4 (bP) into 1 (bP)
with
Graph: 
0: ('P', (True, [1, 2, 3]))
1: ('bP', (True, [0, None, None]))
2: ('bbbP', (True, [0, None, None]))
3: ('abbbP', (True, [0, None, None]))
4: ('bP', (True, [5, 6, 7]))
5: ('bbP', (True, [4

Exception: Not merging

In [46]:
actions = ['a', 'b']
cube_rules = [('aaa', ''),
         ('bbbb', ''),
         ('aP', 'P'),
         ('abP', 'bbbP')]


cube_initial_graph = {
    0: ('P', (True, [1, 3, None])),
    1: ('bP', (True, [2, 0, None])),
    2: ('bbP', (True, [3, 1, None])),
    3: ('bbbP', (True, [0, 2, None])),
}

cube_path = 'aaabababaabab'
common_cycle = common_cycle3
logging.getLogger('').setLevel(logging.ERROR)
progress_graph(cube_initial_graph, cube_path, cube_rules)

{0: ('P', (True, [1, 3, 7])),
 1: ('bP', (True, [0, 14, 2])),
 2: ('bbP', (True, [1, 29, 3])),
 3: ('bbbP', (True, [0, 2, 6])),
 6: ('abbP', (True, [3, 29, 7])),
 7: ('abbbP', (True, [0, 6, 14])),
 14: ('aabbP', (True, [1, 7, 29])),
 29: ('baabbP', (True, [2, 14, 6]))}

In [ ]:
actions = ['a', 'b']
dodca_rules = [('aaa', ''),
         ('bbbbb', ''),
         ('aP', 'P'),
         ('bbbbP', 'abP')]


dodca_initial_graph = {
    0: ('P', (True, [4, 1, None])),
    1: ('bP', (True, [0, 2, None])),
    2: ('bbP', (True, [1, 3, None])),
    3: ('bbbP', (True, [2, 0, None]))
}

path = 'abbabbbb'
common_cycle = common_cycle3
progress_graph(dodca_initial_graph, path, dodca_rules)

START
Applying a
Duplicated graph: 
0: ('P', (True, [4, 1, None]))
1: ('bP', (True, [0, 2, None]))
2: ('bbP', (True, [1, 3, None]))
3: ('bbbP', (True, [2, 0, None]))
4: ('aP', (True, [8, 5, None]))
5: ('abP', (True, [4, 6, None]))
6: ('abbP', (True, [5, 7, None]))
7: ('abbbP', (True, [6, 4, None]))
Reducing 4 : aP into P (aP -> P)
Reduced graph: 
0: ('P', (True, [4, 1, None]))
1: ('bP', (True, [0, 2, None]))
2: ('bbP', (True, [1, 3, None]))
3: ('bbbP', (True, [2, 0, None]))
4: ('P', (True, [8, 5, None]))
5: ('abP', (True, [4, 6, None]))
6: ('abbP', (True, [5, 7, None]))
7: ('abbbP', (True, [6, 4, None]))
Merging the following: {4: 0}
Merging 4 (P) into 0 (P)
with
Graph: 
0: ('P', (True, [4, 1, None]))
1: ('bP', (True, [0, 2, None]))
2: ('bbP', (True, [1, 3, None]))
3: ('bbbP', (True, [2, 0, None]))
4: ('P', (True, [8, 5, None]))
5: ('abP', (True, [4, 6, None]))
6: ('abbP', (True, [5, 7, None]))
7: ('abbbP', (True, [6, 4, None]))
Merging Table: 
{4: 0}
Common cycle of
('P', (True, [8, 5

In [33]:
actions = ['a', 'b']
bucky_rules = [('aaa', ''),
         ('bbbbb', ''),
         ('ababbP', 'bP')]


bucky_initial_graph = {
    0: ('P', (True, [4, 1, 8])),
    1: ('bP', (True, [0, 2, 5])),
    2: ('bbP', (True, [1, 3, None])),
    3: ('bbbP', (True, [2, 4, None])),
    4: ('bbbbP', (True, [3, 0, None])),
    5: ('aaP', (True, [1, None, 6])),
    6: ('aabP', (True, [5, None, 7])),
    7: ('aP', (True, [6, None, 8])),
    8: ('abP', (True, [7, None, 0])),    
}

bucky_path = 'bbbbabbbaaaab'
bucky = progress_graph(bucky_initial_graph, bucky_path, bucky_rules)

In [34]:
actions = ['a', 'b']
bucky_rules = [('aaa', ''),
         ('bbbbb', ''),
         ('ababbP', 'bP')]


bucky_lean_initial_graph = {
    0: ('P', (True, [4, 1, 8])),
    1: ('bP', (True, [0, 2, 5])),
    2: ('bbP', (True, [1, 3, None])),
    3: ('bbbP', (True, [2, 4, None])),
    4: ('bbbbP', (True, [3, 0, None])),
    5: ('aaP', (True, [1, None, 6])),
    6: ('aabP', (True, [5, None, 7])),
    7: ('aP', (True, [6, None, 8])),
    8: ('abP', (True, [7, None, 0])),    
}

bucky_path = 'bbbbabbbaaaab'
progress_graph(bucky_lean_initial_graph, bucky_path, bucky_rules)

{0: ('P', (True, [1, 8, 4])),
 1: ('bP', (True, [0, 2, 5])),
 2: ('bbP', (True, [1, 3, 14])),
 3: ('bbbP', (True, [2, 4, 31])),
 4: ('bbbbP', (True, [0, 65, 3])),
 5: ('aaP', (True, [16, 6, 1])),
 6: ('aabP', (True, [270, 7, 5])),
 7: ('aP', (True, [8, 6, 140])),
 8: ('abP', (True, [0, 7, 134])),
 14: ('baaP', (True, [33, 15, 2])),
 15: ('baabP', (True, [16, 14, 542])),
 16: ('baP', (True, [412, 5, 15])),
 31: ('bbaaP', (True, [32, 3, 67])),
 32: ('bbaabP', (True, [33, 31, 1086])),
 33: ('bbaP', (True, [32, 956, 14])),
 65: ('bbbaaP', (True, [66, 4, 135])),
 66: ('bbbaabP', (True, [65, 2174, 67])),
 67: ('bbbaP', (True, [66, 2044, 31])),
 134: ('bbbbaabP', (True, [8, 139, 135])),
 135: ('bbbbaP', (True, [169, 65, 134])),
 139: ('abbbP', (True, [134, 140, 167])),
 140: ('abbbbP', (True, [201, 139, 7])),
 167: ('abbaaP', (True, [203, 168, 139])),
 168: ('abbaabP', (True, [167, 3262, 169])),
 169: ('abbaP', (True, [168, 2174, 135])),
 201: ('abbbaaP', (True, [202, 140, 271])),
 202: ('abb

In [35]:
progress_graph(bucky_initial_graph, bucky_path, bucky_rules)

{0: ('P', (True, [1, 8, 4])),
 1: ('bP', (True, [0, 2, 5])),
 2: ('bbP', (True, [1, 3, 14])),
 3: ('bbbP', (True, [2, 4, 31])),
 4: ('bbbbP', (True, [0, 65, 3])),
 5: ('aaP', (True, [16, 6, 1])),
 6: ('aabP', (True, [270, 7, 5])),
 7: ('aP', (True, [8, 6, 140])),
 8: ('abP', (True, [0, 7, 134])),
 14: ('baaP', (True, [33, 15, 2])),
 15: ('baabP', (True, [16, 14, 542])),
 16: ('baP', (True, [412, 5, 15])),
 31: ('bbaaP', (True, [32, 3, 67])),
 32: ('bbaabP', (True, [33, 31, 1086])),
 33: ('bbaP', (True, [32, 956, 14])),
 65: ('bbbaaP', (True, [66, 4, 135])),
 66: ('bbbaabP', (True, [65, 2174, 67])),
 67: ('bbbaP', (True, [66, 2044, 31])),
 134: ('bbbbaabP', (True, [8, 139, 135])),
 135: ('bbbbaP', (True, [169, 65, 134])),
 139: ('abbbP', (True, [134, 140, 167])),
 140: ('abbbbP', (True, [201, 139, 7])),
 167: ('abbaaP', (True, [203, 168, 139])),
 168: ('abbaabP', (True, [167, 3262, 169])),
 169: ('abbaP', (True, [168, 2174, 135])),
 201: ('abbbaaP', (True, [202, 140, 271])),
 202: ('abb

In [36]:
bucky_initial_graph

{0: ('P', (True, [4, 1, 8])),
 1: ('bP', (True, [0, 2, 5])),
 2: ('bbP', (True, [1, 3, None])),
 3: ('bbbP', (True, [2, 4, None])),
 4: ('bbbbP', (True, [3, 0, None])),
 5: ('aaP', (True, [1, None, 6])),
 6: ('aabP', (True, [5, None, 7])),
 7: ('aP', (True, [6, None, 8])),
 8: ('abP', (True, [7, None, 0]))}

In [37]:
def find_cycle(g, r, c, d=0):
    if d < 0:
        return 0
    if r in g[c][1]:
        return d
    else:
        return 1 + find_cycle



In [40]:
{k: v for k, v in  [(1, 2), (1, 4)]}

{1: 4}